In [1]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks_cwt
import denoise as dn

In [2]:
with h5py.File('data/deposition.h5', 'r') as source:
    Ebounds = np.array(source['Matrix'].attrs['Edges'])

In [96]:
file_path = 'data/train_2.h5'
event = '1000'

time_interval = 0.1

with h5py.File(file_path, 'r') as source:
    # 对应同一个事件，采用同一个时间段进行衡量
    Time_sup = np.empty((7, 4))
    Time_inf = np.empty((7, 4))
    for cube in range(7):
        for det in range(4):
            Time_sup[cube, det] = source[event]['response_E_T']['D'+str(cube)][str(det)][:,0].max()
            Time_inf[cube, det] = source[event]['response_E_T']['D'+str(cube)][str(det)][:,0].min()
    time_sup = np.floor(Time_sup.min() - time_interval) #每个时刻t的计数的范围对应t~t+dt
    time_inf = np.ceil(Time_inf.max())

    Time = np.arange(time_inf, time_sup, time_interval)
    time_num = Time.size    # 不直接计算，防止float的精度错误
    Tbounds = np.array(Time, Time[-1] + time_interval)

    # 广播规则的预备，对应上下限的准备
    Inf = np.empty((time_num, 100, 2))
    Inf[:,:,0] = np.expand_dims(Time ,1)
    Inf[:,:,1] = np.expand_dims(Ebounds[:100], 0)

    Sup = np.empty((time_num, 100, 2))
    Sup[:,:,0] = np.expand_dims(Time + time_interval ,1)
    Sup[:,:,1] = np.expand_dims(Ebounds[1:], 0)

    # 初始索引顺序：cube, det, time, Eng
    # time, Eng多一个长度用于存储超界的记录
    Counts = np.zeros((7, 4, time_num + 1, 100 + 1))
    for cube in range(7):
        for det in range(4):
            light_doc = source[event]['response_E_T']['D' + str(cube)][str(det)]

            Id_time = (np.expand_dims(light_doc[:, 0], 1) > Tbounds).sum(axis=1) - 1
            Id_time[Id_time == -1] = Tbounds.size - 1
            Id_eng = (np.expand_dims(light_doc[:, 1], 1) > Ebounds).sum(axis=1) - 1
            Id_eng[Id_time == -1] = Ebounds.size - 1

            for i_doc in range(Id_time.size):
                Counts[0, 0, Id_time[i_doc], Id_eng[i_doc]] += 1
            
            '''
            虽然没有用for循环，但下面的广播规则更慢
                对比运行结果为4.3s对0.5s（train_2,事件1000,cube=0,det=0）
                Id_time_eng = np.array([Id_time, Id_eng]).swapaxes(0, 1)
                Id_count = (np.expand_dims(Id_time_eng, 1) == Id_time_eng).prod(axis=2).sum(axis=1)

                Counts[0,0][Id_time, Id_eng] += Id_count
            '''

    # 最终索引顺序：cube, det, eng, time
    # 去除范围外的部分（对应eng, time的最后一项）
    Counts = Counts[:, :, :-1, :-1].swapaxes(2, 3)


#Light_response = np.empty((7, 4, ?, ))



In [4]:
with h5py.File(file_path, 'r') as source:
    a =  source[event]['response_E_T']['D0']['0'][...]


In [ ]:
a < [1,2]

In [6]:
Inf = np.empty((time_num, 100, 2))
Inf[:,:,0] = np.expand_dims(Time ,1)
Inf[:,:,1] = np.expand_dims(Ebounds[:100], 0)

Sup = np.empty((time_num, 100, 2))
Sup[:,:,0] = np.expand_dims(Time + time_interval ,1)
Sup[:,:,1] = np.expand_dims(Ebounds[1:], 0)


In [20]:
Tbounds = np.array(Time, Time[-1] + time_interval)
Id_time = (np.expand_dims(a[:, 0], 1) > Tbounds).sum(axis=1) - 1
Id_time[Id_time == -1] = Tbounds.size - 1
Id_time

array([2239, 2239, 2239, ..., 2239, 2239, 2239])

In [25]:
Id_eng = (np.expand_dims(a[:, 1], 1) > Ebounds).sum(axis=1) - 1
Id_eng[Id_time == -1] = Ebounds.size - 1
Id_eng

array([52,  0,  0, ...,  1,  0,  0])

In [91]:
Counts = np.empty((7, 4, time_num + 1, 100 + 1))

Id_time_eng = np.array([Id_time, Id_eng]).swapaxes(0, 1)
Id_count = (np.expand_dims(Id_time_eng, 1) == Id_time_eng).prod(axis=2).sum(axis=1)

Counts[0,0][Id_time, Id_eng] += Id_count


In [92]:
c = Counts

In [93]:
Counts = np.empty((7, 4, time_num + 1, 100 + 1))
for i in range(Id_time.size):
    Counts[0, 0, Id_time[i], Id_eng[i]] += 1

In [95]:
np.prod(c == Counts)

1

In [51]:
b = np.arange(25)
b = b.reshape(5,5)
b

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [76]:
l1 = np.array([1,2,1,3,5,4,5])
l2 = np.array([1,2,1,2,5,7,5])
l = np.array([l1, l2]).swapaxes(0, 1)
(l[0] == l).prod(axis=1).sum()
(np.expand_dims(l, 1) == l).prod(axis=2).sum(axis=1)

array([2, 1, 2, 1, 2, 1, 2])

In [53]:
b[np.array([1,2,1]), np.array([1,2,1])]

array([ 7, 13,  7])

In [54]:
np.bincount(np.array([1,2,1]))

array([0, 2, 1])